# LES LABIRYNTHES 
## Résolution par Dijkstra

Nous profitons de notre fonction de lecture implémentée précédemment.

In [1]:
def lire_fichier(filepath):
    with open(filepath, 'r') as file: # 'r' pour read/lecture
        # Lire toutes les lignes du fichier
        lignes = file.readlines()
        
        # Extraire les dimensions de la matrice
        dimensions = list(map(int, lignes[0].strip().split()))
        #La méthode .strip(): supprimer les espaces blancs ou des 
        #caractères spécifiques au début et à la fin d'une chaîne de caractères (string).
        
        #la fonction .split(): transformer "10 10 10" en ["10", "10", "10"] 
        
        # Extraire le point d'éntrée
        point1 = tuple(map(int, lignes[1].strip().split()))
        
        # Extraire les point de sortie
        point2 = [tuple(map(int, ligne.strip().split())) for ligne in lignes[2].strip().split(";")]
        #la méthode map va prendre chage élément du tableau donnée par ligne.strip().split() 
        #et appliquer la fonction int(): cast d'une string en entier

        # Lire la matrice
        matrice = []
        for ligne in lignes[3:]: #à partir de la ligne trois jusqu'à la fin
            matrice.append(list(map(int, ligne.strip().split())))
        
        return dimensions, point1, point2, matrice


Nous passons maintenant à l'implémentation de Dijstra. Pour ce faire j'aurai besoin de la classe Node, défini comme suit

In [2]:
class Node():
    #this représente l'objet 
    def __init__(this, parent=None, position=None): #constructeur
        this.preced = parent
        this.position = position
        this.dist = 0

    def __eq__(this, other):                        #surcharge de la fonction de comparaison ==
        return this.position == other.position

    def __lt__(this, other):                     #surcharge de la fonction de comparaison <
        return this.dist < other.dist

Passons maintenant à l'implémentation de la fonction dijkstra

In [3]:
import heapq

def dijkstra(maze, start, exits):
    # Retourne une liste de tuples représentant le chemin du départ à la fin dans le labyrinthe donné

    # Créer les noeuds de départ et de fin
    start_node = Node(None, start)

    # Initialiser la liste ouverte et la liste fermée (ouverte comme une file de priorité)
    open_list = []
    closed_set = set()
    
    # Ajouter le noeud de départ
    heapq.heappush(open_list, start_node)

    best_exit = None  # Meilleure sortie trouvée
    best_end_node = None  # Noeud de la meilleure sortie

    # Boucler jusqu'à trouver la fin
    while open_list:

        # Récupérer le noeud actuel
        current_node = heapq.heappop(open_list)
        closed_set.add(current_node.position)

        # Vérifier si on a atteint une sortie
        if current_node.position in exits:
            if best_exit is None or current_node.dist < best_end_node.dist:
                best_exit = current_node.position
                best_end_node = current_node

        # Générer les voisins
        for voisins in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            voisin_position = (current_node.position[0] + voisins[0], current_node.position[1] + voisins[1])

            if ((voisin_position in closed_set) or #Si déjà exploré, on passe
             not(0 <= voisin_position[0] < len(maze) and 
                0 <= voisin_position[1] < len(maze[0]) and #Si un sommet mur ou en dehors du grid, on passe
                maze[voisin_position[0]][voisin_position[1]] == 0)):
                continue
            
            
            new_node = Node(current_node, voisin_position)
            # Calculate dist
            
            new_node.dist = current_node.dist + 1


            # Regarde si dans la open list et si distance plus petite.
            in_open_list = False
            for open_node in open_list:
                if new_node == open_node and new_node.dist < open_node.dist:
                    open_node = new_node
                    in_open_list = True
                    break
            
            # Regarde si pas dans la open list, y ajouter.
            if not in_open_list:
                heapq.heappush(open_list, new_node)
    return best_end_node, closed_set  # Retourne le meilleur chemin trouvé


Remarquez qu'on fait un ```import heapq``` au début pour charger la bibliotèque ```heapq``` qui nous permet d'utiliser un tas minimal  
Ce tas minimal, organise notre tableau de manière à ce que l'élément au début soit celui avec la plus petite distance.  
Cela est fait à l'aide de la méthode ```_lt_``` qui a été redéfinie dans notre classe  ```Node```

De plus, cette structure nous permet une insertion et suppression à O(nlog n), tout en gardant l'élément avec la plus petite distance au début. 

Ils nous faut maintenant une fonction qui récupère le chemin à partir de la fin:

In [4]:
 
def get_path(end_node):
    path = []
    current_node = end_node
    while current_node is not None:
        path.append(current_node.position)
        current_node = current_node.preced
    # On a le chemin dans le tableau path, mais il est à l'envers
    return path[::-1]  # Returner le chemin contraire


On peut maintenant résoudre notre labirynthe:

In [5]:
fichier = "maze.txt"

dimensions, start, exits, maze = lire_fichier(fichier)

end_node, closed_set = dijkstra(maze, start, exits)
path = get_path(end_node)

for pos in path:
    maze[pos[0]][pos[1]] = 5
for row in maze:
    print(row)

print("nombre de sommets explorés: ",len(closed_set))

[1, 1, 5, 0, 0]
[1, 0, 5, 1, 0]
[1, 1, 5, 0, 0]
[0, 5, 0, 0, 1]
[5, 1, 1, 1, 1]
nombre de sommets explorés:  14


Le chemin est indique par le numéro 5.